In [ ]:
#import import library
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/LOMBA

/content/gdrive/My Drive/LOMBA


In [ ]:
#import dataset
#import dataset
df = pd.read_csv('dataset_street.csv')
df = df.drop(columns="Unnamed: 0")
df.head()

,Sentence,Word,Tag
0,1,jl,B-STR
1,1,kapuk,I-STR
2,1,timur,I-STR
3,1,delta,I-STR
4,1,sili,I-STR


In [ ]:
#Info dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360706 entries, 0 to 2360705
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Sentence  int64 
 1   Word      object
 2   Tag       object
dtypes: int64(1), object(2)
memory usage: 54.0+ MB


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
#mencari kalimat terpanjang dalam dataset
maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 36


In [ ]:
df_dict = pd.read_csv('df_new_dict.csv')
df_dict = df_dict.drop(columns="Unnamed: 0")
df_dict.iloc[0,0]=str(df_dict.iloc[0,0])

In [ ]:
words = df_dict['Words'].values

In [ ]:
n_words = len(words); n_words

87407

In [ ]:
tags = list(set(df["Tag"].values))

In [ ]:
n_tags = len(tags); n_tags

3

In [ ]:
word_list = df_dict['Words'].values
key_list = df_dict['Key'].values
word2idx = dict(zip(word_list, key_list))

In [ ]:
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
tag2idx

{'B-STR': 1, 'I-STR': 0, 'O': 2}

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
print(X[0])

[50845, 15652, 24998, 44692, 53189, 47101, 64955, 26060, 1310, 53476, 17121, 62043, 61479]


In [ ]:
X = pad_sequences(maxlen=36, sequences=X, padding="post",value=87406)
print(X[0])

[50845 15652 24998 44692 53189 47101 64955 26060  1310 53476 17121 62043
 61479 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406]


In [ ]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]
print(y[5])

[1, 0, 0, 2, 2, 2, 2, 2]


In [ ]:
y = pad_sequences(maxlen=36, sequences=y, padding="post", value=tag2idx["O"])
print(y)

[[1 0 0 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 1 2 ... 2 2 2]
 ...
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]]


In [ ]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
print(y[0])

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model_street.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.01)

In [ ]:
Y = np.array(y)

In [ ]:
from sklearn.model_selection import KFold
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
test_loss = 1
for train, test in kfold.split(X, Y):
  # create model
  input=Input(shape=(36,))
  model=Embedding(input_dim=n_words, output_dim=36, input_length=36)(input)
  model=Dropout(0.2)(model)
  model=Bidirectional(LSTM(units=117, return_sequences=True, activity_regularizer=l2(0.01), recurrent_dropout=0.2))(model)
  out=TimeDistributed(Dense(n_tags, activation="softmax"))(model)
  model=Model(input, out)
	# Compile model
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
	# Fit the model
  model.fit(X[train], Y[train], batch_size=32, epochs=5, validation_split=0.2, verbose=1, callbacks=[mc,es])
	# evaluate the model
  scores = model.evaluate(X[test], Y[test], verbose=1)
  if test_loss>scores[0]:
    test_loss=scores[0]
    model.save_weights('best_model_cv_final_street.h5')
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/5
6000/6000 [==============================] - 1067s 177ms/step - loss: 0.1024 - accuracy: 0.9767 - val_loss: 0.0439 - val_accuracy: 0.9856
Epoch 2/5
6000/6000 [==============================] - 1048s 175ms/step - loss: 0.0397 - accuracy: 0.9869 - val_loss: 0.0407 - val_accuracy: 0.9861
Epoch 3/5
6000/6000 [==============================] - 1043s 174ms/step - loss: 0.0340 - accuracy: 0.9889 - val_loss: 0.0433 - val_accuracy: 0.9861
Epoch 00003: early stopping
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0433 - accuracy: 0.9861
accuracy: 98.61%
Epoch 1/5
6000/6000 [==============================] - 1048s 174ms/step - loss: 0.0912 - accuracy: 0.9743 - val_loss: 0.0437 - val_accuracy: 0.9855
Epoch 2/5
6000/6000 [==============================] - 1044s 174ms/step - loss: 0.0392 - accuracy: 0.9871 - val_loss: 0.0415 - val_accuracy: 0.9858
Epoch 3/5
6000/6000 [==============================] - 1042s 174ms/step - loss: 0.0340 - accuracy: 0.9889 - val_loss: 0.04

In [ ]:
input=Input(shape=(36,))
model=Embedding(input_dim=n_words, output_dim=36, input_length=36)(input)
model=Dropout(0.2)(model)
model=Bidirectional(LSTM(units=117, return_sequences=True, activity_regularizer=l2(0.01), recurrent_dropout=0.2))(model)
out=TimeDistributed(Dense(n_tags, activation="softmax"))(model)
model=Model(input, out)

In [ ]:
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 36, 36)            3146616   
_________________________________________________________________
dropout (Dropout)            (None, 36, 36)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 36, 234)           144144    
_________________________________________________________________
time_distributed (TimeDistri (None, 36, 3)             705       
Total params: 3,291,465
Trainable params: 3,291,465
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=3, validation_split=0.2, verbose=1, callbacks=[es,mc])

Epoch 1/3
6000/6000 [==============================] - 546s 91ms/step - loss: 0.1698 - accuracy: 0.9720 - val_loss: 0.0502 - val_accuracy: 0.9857
Epoch 2/3
6000/6000 [==============================] - 539s 90ms/step - loss: 0.0404 - accuracy: 0.9883 - val_loss: 0.0416 - val_accuracy: 0.9859
Epoch 3/3
6000/6000 [==============================] - 535s 89ms/step - loss: 0.0291 - accuracy: 0.9911 - val_loss: 0.0418 - val_accuracy: 0.9862
Epoch 00003: early stopping


In [ ]:
model.load_weights('best_model_cv_final_street.h5')

In [ ]:
model.evaluate(X_test,np.array(y_test))

1875/1875 [==============================] - 38s 20ms/step - loss: 0.0323 - accuracy: 0.9894


[0.03237496316432953, 0.9892712831497192]

In [ ]:
from tqdm import tqdm

pred_ = []
true_ = []
id_test = range(len(X_test))
for i in tqdm(id_test):
  p = model.predict(np.array([X_test[i]]))
  p = np.argmax(p, axis=-1)
  p_trans = np.transpose(p)
  count = 0
  for x in p_trans :
    if x == tag2idx["I-STR"] :
      if p_trans[count-1] == tag2idx["O"] :
        p_trans[count-1] = tag2idx["B-STR"]
        count += 1
      else :
        count += 1
    else :
      count += 1
    p_fin = np.transpose(p_trans)
  q = np.array(y_test[i])
  q = np.argmax(q, axis=-1)
  pred_.append(p_fin[0])
  true_.append(q)

100%|██████████| 60000/60000 [47:38<00:00, 20.99it/s]


In [ ]:
true_temp = np.array(true_)
pred_temp = np.array(pred_)

In [ ]:
analyze = []
true_score=[]
dummy = 0
for i in tqdm(id_test):
  for j in range(0,36):
    if true_temp[i,j]==pred_temp[i,j]:
      dummy += 1
    else :
      analyze.append(0)
      dummy = 0
      break
  if dummy == 36 :
    analyze.append(1)
    true_score.append(1)
    dummy = 0
  else :
    dummy = 0


print("acc : ",len(true_score)/len(id_test)*100,"%")

100%|██████████| 60000/60000 [00:01<00:00, 53086.21it/s]

acc :  79.81166666666667 %


In [ ]:
ready_poi = []
for i in tqdm(id_test):
  ans = ''
  for w,pred in zip(X_test[i],pred_[i]):
    if tags[pred] == "B-STR":
      ans = ans + str(words[w])
    elif tags[pred] == "I-STR":
      ans = ans + ' ' + str(words[w])
    else:
      ans = ans
  ready_poi = np.append(ready_poi,ans)

100%|██████████| 60000/60000 [02:30<00:00, 398.86it/s]


RUN THIS AFTER PREDICTING X_TEST

In [ ]:
test = pd.read_csv("df_new.csv")
data_test = test.drop(['Unnamed: 0'],axis=1)
data_test

,Sentence,Word
0,1,s
1,1,dot
2,1,par
3,1,53
4,1,sidanegara
...,...,...
392702,49999,no
392703,49999,dot
392704,49999,11
392705,50000,adi


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w) for w in zip(s["Word"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter_test = SentenceGetter(data_test)
sentences_test = getter_test.sentences
maxlen = max([len(s) for s in sentences_test])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 33


In [ ]:
X_fintest = [[word2idx[w[0]] for w in s] for s in sentences_test]
X_fintest = pad_sequences(maxlen=36, sequences=X_fintest, padding="post",value=87406)
print(X_fintest[0])

[73351 16517 26887 24589 33464 30505 77919 43143 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406
 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406 87406]


In [ ]:
from tqdm import tqdm

pred_ = []
true_ = []
id_test = range(len(X_fintest))
for i in tqdm(id_test):
  p = model.predict(np.array([X_fintest[i]]))
  p = np.argmax(p, axis=-1)
  p_trans = np.transpose(p)
  count = 0
  for x in p_trans :
    if x == tag2idx["I-STR"] :
      if p_trans[count-1] == tag2idx["O"] :
        p_trans[count-1] = tag2idx["B-STR"]
        count += 1
      else :
        count += 1
    else :
      count += 1
    p_fin = np.transpose(p_trans)
  pred_.append(p_fin[0])

100%|██████████| 50000/50000 [39:45<00:00, 20.96it/s]


In [ ]:
pred_[6]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
print("{:14} ({:5}): {}".format("Word", "True", "Pred"))
for w,pred in zip(X_fintest[12],pred_[12]):
    print("{:14}: {}".format(words[w],tags[pred]))

Word           (True ): Pred
kimia         : O
farma         : O
comma         : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O


In [ ]:
poi_pred_final=[]
for i in words:
  word_=str(i).replace(" dot ",". ")
  word_=word_.replace(" dot","")
  word_=word_.replace("dot ","")
  word_=word_.replace(" comma ",", ")
  word_=word_.replace("comma ","")
  word_=word_.replace(" comma","")
  poi_pred_final.append(word_)

In [ ]:
ready_poi = []
for i in tqdm(id_test):
  ans = ''
  for w,pred in zip(X_fintest[i],pred_[i]):
    if tags[pred] == "B-STR":
      ans = ans + str(poi_pred_final[w]) + ' '
    elif tags[pred] == "I-STR":
      ans = ans + str(poi_pred_final[w]) + ' '
    else:
      ans = ans
  if ans!="":
    ready_poi = np.append(ready_poi,ans[:-1])
  else:
    ready_poi = np.append(ready_poi,ans)

100%|██████████| 50000/50000 [01:38<00:00, 506.83it/s]


In [ ]:
ready_poi[0:100]

array(['s dot par', 'angg per', 'mand imog', 'raya nga', 'cut mutia',
       'perum gar', '', '', '', 'raya won', 'tebet raya', 'kemur vii', '',
       '', 'raya pan jakat', 'raya', 'jl tanjung ii', 'ayu 3',
       'pasar pagi raya', 'sukam', 'gg dot damai 1', 'ser aswan',
       'gg dot pan', 'tebet barat vi', 'gad kir timur vi', '',
       'raya bant', 'pulo mas barat vi', '', 'ki ageng peman',
       'ahmad yani 19', 'jln dot hbr motik', 'gg dot mah', 'pab',
       'star barat ii', 'taman cib', 'm dot t dot hary', 'ged hijau vii',
       '', 'raya setu cibi bek', 'jl abu 1', '', 'letj supr',
       'mand sela ii', 'jl jl dot letjen sutoyo', '', 'paru',
       'pasar raya', 'pon', 'vete', 'raden saleh raya', 'bhin karya', '',
       'mahk rege', 'jend sudi', 'taman hara baru', 'aka delp',
       'kh dot wahid has', '', 'raya kapas', 'pal xi', 'cibi', 'mast',
       'kawat i', 'lele', 'jamba', 'perw', 'pulau bida', 'par sarpo i',
       'desa ken', '', '', 'derm lonc', '', 'mla ii', '

In [ ]:
poi_pred_final=[]
for i in ready_poi:
  word_=i.replace(" dot ",". ")
  word_=word_.replace(" dot","")
  word_=word_.replace("dot ","")
  word_=word_.replace(" comma ",", ")
  word_=word_.replace("comma ","")
  word_=word_.replace(" comma","")
  poi_pred_final.append(word_)

In [ ]:
poi_df = pd.DataFrame(poi_pred_final, columns = ['Street'])

In [ ]:
poi_df.head()

,Street
0,s. par
1,angg per
2,mand imog
3,raya nga
4,cut mutia


In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
false_alarm=[]
poi_word=[]
idx=[]
for i in range(len(poi_df)):
  if poi_df.iloc[i,0]!="":
    if poi_df.iloc[i,0] not in df_test.iloc[i,1]:
      idx.append(i)
      false_alarm.append(poi_df.iloc[i,0])
      poi_word.append(df_test.iloc[i,1])

In [ ]:
len(false_alarm)

41

In [ ]:
i = 17
print(false_alarm[i],"--",poi_word[i])

terr avenue pis kipas -- terr avenue., pis kipas jati mulyo


In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
#string1 raw string 2 poi/street
def find_keyword(string1,string2):
  keyword = ""
  string1 = string1.split()
  for i in string1:
    if i!='' and fuzz.ratio(i,string2) >= 70 and len(keyword.split())<len(string2.split()):
      keyword+=i+" "
  return(keyword[:-1])

In [ ]:
false_alarm[101:111]

['kavl al i',
 'gg. mush at taqwa 1',
 'jl. raya jl. bunga rampe 88kel',
 'raya sore banj',
 'jepara bangsri',
 'jl. teknologi xii',
 'i. bou bont',
 'raya ngawi caru',
 'gg. h. m. ma i',
 'manu lor ii a']

In [ ]:
true_poi=[]
for i,j in zip(false_alarm,poi_word):
  start_i=i.split()[0]
  end_i=i.split()[-1]
  #print(end_i,"----")
  poi_w = j.split()
  start_con=False
  poi_w_=""
  for k in poi_w:
    if find_keyword(k,start_i)==k:
      start_con=True
    if start_con:
      poi_w_+=k+' '
    if find_keyword(k,end_i)==k:
      start_con=False
  true_poi.append(poi_w_[:-1])

In [ ]:
true_poi[0:10]

['jln.hbr motik',
 'raya setu cibi - bek',
 'jl. karya baru jl. letjen sutoyo,',
 'jl.a dah',
 'ngareanak pat - boja',
 '',
 'jend r.s. soek,',
 '',
 'kop sang cab. maum gajah mada',
 "tanj sa'ar 95 jatiluhur jatiasih"]

In [ ]:
idx[0:10]

[31, 39, 44, 102, 206, 246, 255, 286, 347, 506]

In [ ]:
poi_df.iloc[idx,0]=true_poi

In [ ]:
poi_df.iloc[idx,0]

31                           jln.hbr motik
39                    raya setu cibi - bek
44       jl. karya baru jl. letjen sutoyo,
102                               jl.a dah
206                   ngareanak pat - boja
                       ...                
49837                         al - mansyu,
49839                                     
49902                           jl.t merd,
49927                                     
49943                     poros maj - mam,
Name: Street, Length: 1246, dtype: object

In [ ]:
poi_df.to_csv('street_test.csv')
!cp 'street_test.csv' "/content/gdrive/My Drive/LOMBA"

cp: 'street_test.csv' and '/content/gdrive/My Drive/LOMBA/street_test.csv' are the same file


In [ ]:
poifin = pd.read_csv("poi_test.csv")
streetfin = pd.read_csv("street_test.csv")

poi_proc = poifin["POI"]
street_proc = streetfin["Street"]

df1 = poi_proc.replace(np.nan, '', regex=True)
df2 = street_proc.replace(np.nan, '', regex=True)

In [ ]:
df2[0:10]

0       s. par
1     angg per
2    mand imog
3     raya nga
4    cut mutia
5    perum gar
6             
7             
8             
9     raya won
Name: Street, dtype: object

In [ ]:
fin_str = []
for x in tqdm(range(len(df1))):
  str_add = str(df1[x]) + "/" + str(df2[x])
  fin_str = np.append(fin_str,str_add)

100%|██████████| 50000/50000 [02:59<00:00, 278.85it/s]


In [ ]:
fin_submit = pd.DataFrame({'POI/street':fin_str})
fin_submit.insert(0, 'id', range(len(df1)))

In [ ]:
fin_submit

,id,POI/street
0,0,/s. par
1,1,/angg per
2,2,asma laun/mand imog
3,3,ud agung/raya nga
4,4,/cut mutia
...,...,...
49995,49995,toko mbak farid/
49996,49996,tkridho kids/vete
49997,49997,mart dan roti bakar malabar/nasio
49998,49998,graha indah pamulang/jl. mujair


In [ ]:
fin_submit.to_csv('submit.csv', index = False)